In [1]:
# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json
import random


In [3]:
from geopy.distance import geodesic

# Tọa độ điểm 1
point1 = (10.79787015, 106.5972391)  # Tọa độ (vĩ độ, kinh độ)

# Tọa độ điểm 2
point2 = (10.82609876, 106.6206312)  # Tọa độ (vĩ độ, kinh độ)

# Tính khoảng cách
distance = geodesic(point1, point2).kilometers

print(f"Khoảng cách giữa hai điểm là: {distance:.2f} km")


Khoảng cách giữa hai điểm là: 4.04 km


In [2]:
scraper = cloudscraper.create_scraper(
            browser={
                "browser": "chrome",
                "platform": "windows",
            },
        )

# specify the target URL

url = f"https://www.topcv.vn/viec-lam/quan-ly-chi-nhanh-trung-tam-tieng-anh/1515959.html"
# url = 'https://www.topcv.vn/viec-lam/ky-su-kinh-doanh-thiet-bi-moi-truong-tp-hcm-luong-cung-15-000-000-vnd/1367302.html'
# time.sleep(2)
response = scraper.get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')


200


In [3]:
def get_name_company_input(soup):
    company_name = soup.find(['h1' ,'h2', 'h3'], class_="company-name-label")
    if company_name:
        name = company_name.get_text(separator=' ', strip=True)
        return name
    return ""

In [4]:
def get_heading_input(soup):
    heading1 = soup.find('h1', class_="job-detail__info--title")
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return str(content)
    return ""

In [5]:
def get_meta_description_input(soup):
    data = soup.find('meta', attrs={"name": "description"})
    if data:
        return str(data.get('content').strip())
    return ""

In [6]:
def thongTinChung_input(soup):
    # Danh sách các tiêu đề cần tìm
    titles_to_find = ["Cấp bậc", "Số lượng tuyển", "Hình thức làm việc", "Giới tính"]
    array = []
    # Duyệt qua từng tiêu đề và tìm giá trị tương ứng
    for title in titles_to_find:
        title_div = soup.find('div', class_="box-general-group-info-title", string=title)
        if title_div:
            value_div = title_div.find_next('div', class_="box-general-group-info-value")
            if value_div:
                value = value_div.text.strip()
                array.append(value)
    if not array:
        return ""
    if array[3].lower() == "Không yêu cầu".lower():
        array[3] = "không yêu cầu về giới tính"
    else: 
        array[3] =  'với giới tính là ' + array[3]
    return array

In [7]:
def thongTinCongViec_input(soup):
    titles_to_find = ["Mô tả công việc", "Yêu cầu ứng viên", "Quyền lợi"]
    array = []
    for title in titles_to_find:
        # Tìm thẻ <h3> với nội dung "Mô tả công việc"
        job_desc_title = soup.find(['h2', 'h3'], string=title)
        if job_desc_title:
            content_div = job_desc_title.find_next('div', class_='job-description__item--content')
            if content_div:
                # Lấy toàn bộ nội dung HTML bên trong, loại bỏ thẻ <div> ngoài cùng
                # inner_html = content_div.text.strip()
                inner_html = "\n".join(content_div.stripped_strings)
                array.append(inner_html)
            else:
                array.append(None)
    if not array:
        return ""
    return array

In [8]:
def dia_chi_input(soup):
    job_desc_title = soup.find(['h2', 'h3'], string="Địa điểm làm việc")
    content_div = job_desc_title.find_next('div', class_='job-description__item--content')
    if content_div:
    # Tìm thẻ <a> với id="toggle-more-job-addresses"
        a_tag = content_div.find('a', id="toggle-more-job-addresses")
        if a_tag:  # Nếu tìm thấy thẻ <a>
            a_tag.decompose()  # Xóa thẻ <a>
        # div_tag = content_div.find('div', id="more-job-addresses")
        # if div_tag:  # Nếu tìm thấy thẻ <a>
        #     div_tag.decompose()  # Xóa thẻ <
        job_desc = content_div.text.strip()
        return job_desc
    return ""

In [9]:
def thoiGianLamViec_input(soup):
    # Tìm thẻ <h3> với nội dung "Mô tả công việc"
    job_desc_title = soup.find(['h2', 'h3'], string='Thời gian làm việc')

    # Nếu tìm thấy, lấy thông tin mô tả công việc từ thẻ sau đó
    if job_desc_title:
        content_div = job_desc_title.find_next('div', class_='job-description__item--content')
        if content_div:
            job_desc = content_div.get_text(separator='\n', strip=True)  # Dùng separator để ngăn cách các câu
            return job_desc
    return ""

In [10]:
def hanNopHoSo_input(soup):
    deadline_div = soup.find('div', class_='job-detail__info--deadline')
    # Nếu tìm thấy, lấy phần thông tin về hạn nộp hồ sơ
    if deadline_div:
        deadline = deadline_div.text.strip()
        deadline = deadline.lstrip("Hạn nộp hồ sơ: ").strip()
        return str(deadline)
    else:
        deadline_div = soup.find('div', class_='job-detail__information-detail--actions-label is-expired')
        if deadline_div:
            deadline = deadline_div.text.strip()
            return str(deadline)
    return ""

In [11]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [12]:
def get_heading(soup):
    heading1 = soup.find('h1', class_="job-detail__info--title")
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return str(content)
    return None

In [13]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return str(canonical_url)
    return None

In [14]:
def thongTinChung(soup):
    # Danh sách các tiêu đề cần tìm
    titles_to_find = ["Cấp bậc", "Số lượng tuyển", "Hình thức làm việc", "Giới tính"]
    array = []
    # Duyệt qua từng tiêu đề và tìm giá trị tương ứng
    for title in titles_to_find:
        title_div = soup.find('div', class_="box-general-group-info-title", string=title)
        if title_div:
            value_div = title_div.find_next('div', class_="box-general-group-info-value")
            if value_div:
                value = value_div.text.strip()
                array.append(value)
    if not array:
        return None
    return array

In [15]:
def thongTinCongViec(soup):
    titles_to_find = ["Mô tả công việc", "Yêu cầu ứng viên", "Quyền lợi"]
    array = []
    for title in titles_to_find:
        # Tìm thẻ <h3> với nội dung "Mô tả công việc"
        job_desc_title = soup.find(['h2', 'h3'], string=title)
        if job_desc_title:
            content_div = job_desc_title.find_next('div', class_='job-description__item--content')
            if content_div:
                # Lấy toàn bộ nội dung HTML bên trong, loại bỏ thẻ <div> ngoài cùng
                # inner_html = content_div.text.strip()
                inner_html = list(content_div.stripped_strings)
                array.append(inner_html)
            else:
                array.append(None)
    if not array:
        return None
    return array

In [16]:
def dia_chi(soup):
    output = []
    job_desc_title = soup.find(['h2', 'h3'], string="Địa điểm làm việc")
    content_div = job_desc_title.find_next('div', class_='job-description__item--content')
    if content_div:
    # Tìm thẻ <a> với id="toggle-more-job-addresses"
        a_tag = content_div.find('a', id="toggle-more-job-addresses")
        if a_tag:  # Nếu tìm thấy thẻ <a>
            a_tag.decompose()  # Xóa thẻ <a>
        # div_tag = content_div.find('div', id="more-job-addresses")
        # if div_tag:  # Nếu tìm thấy thẻ <a>
        #     div_tag.decompose()  # Xóa thẻ <
        job_desc = content_div.text.strip()
        output = job_desc.split('\n')
        return output
    return None

In [17]:
def thoiGianLamViec(soup):
    output = []
    # Tìm thẻ <h3> với nội dung "Mô tả công việc"
    job_desc_title = soup.find(['h2', 'h3'], string='Thời gian làm việc')

    # Nếu tìm thấy, lấy thông tin mô tả công việc từ thẻ sau đó
    if job_desc_title:
        content_div = job_desc_title.find_next('div', class_='job-description__item--content')
        if content_div:
            job_desc = content_div.get_text(separator='\n', strip=True)  # Dùng separator để ngăn cách các câu
            output = job_desc.split('\n')
            return output
    return None

In [18]:
def hanNopHoSo(soup):
    deadline_div = soup.find('div', class_='job-detail__info--deadline')
    # Nếu tìm thấy, lấy phần thông tin về hạn nộp hồ sơ
    if deadline_div:
        deadline = deadline_div.text.strip()
        deadline = deadline.lstrip("Hạn nộp hồ sơ: ").strip()
        return str(deadline)
    else:
        deadline_div = soup.find('div', class_='job-detail__information-detail--actions-label is-expired')
        if deadline_div:
            deadline = deadline_div.text.strip()
            return str(deadline)
    return None

In [19]:
def get_name_company(soup):
    company_name = soup.find(['h1' ,'h2', 'h3'], class_="company-name-label")
    if company_name:
        name = company_name.get_text(separator=' ', strip=True)
        return name
    return None

In [20]:
def thongTinTop(soup):
    # Danh sách các tiêu đề cần tìm
    titles_to_find = ["Mức lương", "Địa điểm", "Kinh nghiệm"]
    array = []
    # Duyệt qua từng tiêu đề và tìm giá trị tương ứng
    for title in titles_to_find:
        title_div = soup.find('div', class_='job-detail__info--section-content-title', string=title)
        if title_div:
            value_div = title_div.find_next('div', class_="job-detail__info--section-content-value")
            if value_div:
                value = value_div.text.strip()
                array.append(value)
    if not array:
        return None
    return array

In [21]:
def thongTinLienQuan(soup):
    titles_to_find = ["Danh mục Nghề liên quan", "Khu vực", 'Kỹ năng cần có', 'Kỹ năng nên có']
    dict = {}
    for title in titles_to_find:
        array = []
        jobs = soup.find('div', class_="box-title", string=title)
        if jobs:
            jobs_div = jobs.find_next('div', class_='box-category-tags')
            if jobs_div:
                # Lấy nội dung của các nghề và loại bỏ khoảng trắng thừa
                jobs = jobs_div.get_text(separator='; ', strip=True).split(';')
                array = array + jobs 
        dict[convert_to_camel_case(title)] = array
    if not dict:
        return None
    return dict

In [22]:
def process_ChuyenMon(text):
    if not text:
        return None
    data =  re.sub(r"^chuyên môn[: ]+", "", text, flags=re.IGNORECASE)
    data_split = data.split(';')
    return str(data_split[0])


In [23]:
def linhVuc(soup):
    jobs = soup.find('div', class_="job-tags")
    if jobs:
        # Tách nội dung các phần tử con trong 'job-tags' và ngăn cách bằng dấu ";"
        job_list = jobs.get_text(separator='; ', strip=True)
        return str(process_ChuyenMon(job_list))
    return None

In [24]:
# import numpy as np
# # Hàm để xử lý kiểu dữ liệu không serializable
# def default_converter(o):
#     if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
#         return int(o)
#     elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
#         return o.tolist()
#     return str(o)  # Xử lý các kiểu dữ liệu khác

In [25]:
# # Đọc file JSON
# with open("data_2_12_sample_baiViet_topCV_new1.json", "r", encoding="utf-8") as file:
#     data = json.load(file)

In [26]:
# danh_sach = []
# for temp in data:
#     danh_sach.append(temp.get('urljob'))

In [27]:
# result = []
# found = False
# for temp in danh_sach:
#     if '1367783' in temp:
#         found = True  # Đánh dấu đã tìm thấy
#     if found:  # Nếu đã tìm thấy, thêm phần tử vào kết quả
#         result.append(temp)

In [28]:
def get_id_company(url):
    if url:
        # Regex
        pattern = r"\/(\d+)(?=\.html)"
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

In [35]:
import random
import requests
# 1369794, step 2 1371126
# 1369795 , step 2, 1371097
array_input = []
array_output = []
# link_cong_ty_save = []
for temp in range(0,100000,1):
# for temp in result:
    while True:
        try:
            ids = 1492061 - temp 
            scraper = cloudscraper.create_scraper(
                browser={
                    "browser": "chrome",
                    "platform": "windows",
                },
            )

            # specify the target URL

            url = f"https://www.topcv.vn/viec-lam/pho-bo-phan-thiet-ke-kien-truc-noi-that/{ids}.html"
            # url = temp
            # time.sleep(2)
            response = scraper.get(url)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # print(thong_tin_top)
                if get_heading(soup):
                    data_output = {
                        'url': get_url(soup),
                        'JobTitle': get_heading(soup),
                        'CompanyName': get_name_company(soup),
                        'NgheLienQuan': thongTinLienQuan(soup).get('DanhMucNgheLienQuan'),
                        'KyNang': thongTinLienQuan(soup).get('KyNangCanCo'),
                        'MucLuong': thongTinTop(soup)[0],
                        'KinhNghiem': thongTinTop(soup)[2],
                        'SoLuong': thongTinChung(soup)[1],
                        'LoaiCongViec': thongTinChung(soup)[2],
                        'GioiTinh': thongTinChung(soup)[3],
                        'MoTa': thongTinCongViec(soup)[0],
                        'YeuCau': thongTinCongViec(soup)[1],
                        'QuyenLoi': thongTinCongViec(soup)[2],
                        'DiaChi': dia_chi(soup),
                        'ThoiGianLamViec': thoiGianLamViec(soup),
                        'HanNopHoSo': hanNopHoSo(soup)
                    }
                    data_input = get_heading_input(soup) + '\n' + get_name_company(soup) + ' tuyển ' + get_heading_input(soup) + ' với số lượng là ' + thongTinChung_input(soup)[1] +'. Hình thức làm việc ' + thongTinChung_input(soup)[2] + ', ' + thongTinChung_input(soup)[3] + '. \nVới mức lương: '+ thongTinTop(soup)[0] + '\n' + 'Mô tả công việc' + '\n' + thongTinCongViec_input(soup)[0] + '\n' + 'Yêu cầu ứng viên' + '\n' + thongTinCongViec_input(soup)[1] + '\n' + 'Quyền lợi' + '\n' + thongTinCongViec_input(soup)[2] + '\n' + 'Địa điểm làm việc' + '\n' + dia_chi_input(soup) + '\n' + 'Thời gian làm việc' + '\n' + thoiGianLamViec_input(soup) + '\n' + 'Hạn nộp hồ sơ: ' + hanNopHoSo_input(soup)
                    # data_input = get_heading_input(soup) + '\n' + get_name_company(soup) + ' tuyển ' + get_heading_input(soup)+  ' với số lượng là ' + thongTinChung_input(soup)[1] + '. Hình thức làm việc ' + thongTinChung_input(soup)[2] + ', ' + thongTinChung_input(soup)[3] + '.' + '\n' + 'Mô tả công việc' + '\n' + thongTinCongViec_input(soup)[0] + '\n' + 'Yêu cầu ứng viên' + '\n' + thongTinCongViec_input(soup)[1] + '\n' + 'Quyền lợi' + '\n' + thongTinCongViec_input(soup)[2] + '\n' + 'Địa điểm làm việc' + '\n' + dia_chi_input(soup) + '\n' + 'Thời gian làm việc' + '\n' + thoiGianLamViec_input(soup) + '\n' + 'Hạn nộp hồ sơ: ' + hanNopHoSo_input(soup)
                    array_input.append(data_input)
                    # print(data_input)
                    array_output.append(data_output)
                    print(response.status_code, "Cập nhật thông tin thành công.", get_id_company(url), len(array_output))
            if response.status_code == 429:
                # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {get_id_company(url)}: Lỗi kết nối {e}. Đang thử lại...")
            # time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại
        except (requests.exceptions.ChunkedEncodingError, requests.exceptions.Timeout) as e:
            print(f"ID {get_id_company(url)}: Lỗi kết nối {e}. Đang thử lại...")
            # time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại
            


200
200 Cập nhật thông tin thành công. 1492061 1
200
200
200 Cập nhật thông tin thành công. 1492059 2
200
200 Cập nhật thông tin thành công. 1492058 3
200
200
200 Cập nhật thông tin thành công. 1492056 4
200
200 Cập nhật thông tin thành công. 1492055 5
200
200
200
200 Cập nhật thông tin thành công. 1492052 6
200
200 Cập nhật thông tin thành công. 1492051 7
200
200 Cập nhật thông tin thành công. 1492050 8
200
200 Cập nhật thông tin thành công. 1492049 9
200
200 Cập nhật thông tin thành công. 1492048 10
200
200 Cập nhật thông tin thành công. 1492047 11
200
200
200 Cập nhật thông tin thành công. 1492045 12
200
200 Cập nhật thông tin thành công. 1492044 13
200
200
200
200
200 Cập nhật thông tin thành công. 1492040 14
200
200 Cập nhật thông tin thành công. 1492039 15
200
200 Cập nhật thông tin thành công. 1492038 16
200
200 Cập nhật thông tin thành công. 1492037 17
200
200 Cập nhật thông tin thành công. 1492036 18
200
200 Cập nhật thông tin thành công. 1492035 19
200
200 Cập nhật thông tin 

KeyboardInterrupt: 

In [41]:
array_new = []
for element in array_output:
        json_string = json.dumps(element, ensure_ascii=False)
        array_new.append(json_string)

In [43]:
array_new[0]

'{"url": "https://www.topcv.vn/viec-lam/nhan-vien-van-phong-uu-tien-sv-moi-ra-truong/1492061.html", "JobTitle": "Nhân Viên Văn Phòng (Ưu Tiên SV Mới Ra Trường)", "CompanyName": "CÔNG TY TNHH ĐẦU TƯ VÀ DỊCH VỤ TN GROUP", "NgheLienQuan": ["Kinh doanh/Bán hàng", " Kinh doanh/Bán hàng khác", " Chuyên môn Kinh doanh/Bán hàng khác"], "KyNang": ["Kỹ năng giao tiếp", " Làm việc nhóm"], "MucLuong": "8 - 15 triệu", "KinhNghiem": "Không yêu cầu kinh nghiệm", "SoLuong": "5 người", "LoaiCongViec": "Toàn thời gian", "GioiTinh": "Không yêu cầu", "MoTa": ["Nhận cuộc gọi từ khách hàng liên hệ đến có nhu cầu về sản phẩm Công ty (Khách hàng Hotline).", "Giải quyết những thắc mắc & nhu cầu của Khách hàng về các sản phẩm của Công ty", "(Được đào tạo kiến thức từ a-z).", "Tư vấn, telesale khách hàng theo nguồn Data: Công ty cung cấp, Nguồn khách marketing Công ty phân xuống, Mối quan hệ,...", "Chăm sóc, hỗ trợ khách hàng mới và khách hàng cũ.", "Hỗ trợ viết hóa đơn, hợp đồng khách hàng khi yêu cầu.", "Thực 

In [44]:
import pandas as pd

# Hai danh sách
# list1 = ["Kinh doanh", "Bảo hiểm", "Bất động sản", "Công nghệ thông tin"]
# list2 = [50, 40, 30, 60]  # Số lượng hoặc giá trị tương ứng

# Tạo DataFrame với hai cột
df = pd.DataFrame({
    "jd_input": array_input,
    "jd_output": array_new,
})

# Lưu DataFrame vào file Excel
df.to_excel("data_jd_for_ner_final12.xlsx", index=False, engine="xlsxwriter")

print("Dữ liệu đã được lưu vào file 'data_jd_for_ner12.xlsx'")


Dữ liệu đã được lưu vào file 'data_jd_for_ner12.xlsx'


In [378]:
json.dumps(array_output[5], ensure_ascii=False)

'{"url": "https://www.topcv.vn/viec-lam/nhan-vien-kiem-soat-noi-bo/1367792.html", "JobTitle": "Nhân Viên Kiểm Soát Nội Bộ", "CompanyName": "CÔNG TY TNHH ĐẦU TƯ VÀ PHÁT TRIỂN THƯƠNG MẠI QUỐC CƯỜNG", "NgheLienQuan": ["Hành chính / Văn phòng", " Kế toán / Kiểm toán", " Quản lý chất lượng (QA/QC)"], "KyNang": [], "MucLuong": "10 - 15 triệu", "KinhNghiem": "3 năm", "SoLuong": "1 người", "LoaiCongViec": "Toàn thời gian", "GioiTinh": null, "MoTa": ["-Thực hiện kiểm tra, giám sát xử lý các sự cố, thanh kiểm tra các hoạt động liên quan đến chất lượng, hiệu quả, hiệu suất, tiến độ triển khai, kiểm soát deadline công việc các bộ phận, tính tuân thủ chính sách, các giao dịch kinh doanh, các bộ phận và nền tảng sử dụng trong sản xuất kinh doanh", "-Kiểm soát đội ngũ nhân sự,.. thực hiện đúng quy trình/quy định của công ty", "-Báo cáo, để xuất các cơ chế gửi các bộ phận thực thi.", "-Đảm bảo chất lượng, nâng cao dịch vụ, tham gia xây dựng cải tiến quy định, quy trình của công ty."], "YeuCau": ["-Tốt

In [386]:
json_array_output = []
for temp in array_output:
    temp1 = json.dumps(temp, ensure_ascii=False)
    json_array_output.append(temp1)

In [383]:
for temp in array_output:
    temp1 = json.dumps(temp, ensure_ascii=False)
    print(json.loads(temp1).get('url'))

https://www.topcv.vn/viec-lam/content-tiktok-san-pham-sach/1367783.html
https://www.topcv.vn/viec-lam/chuyen-vien-devops/1367784.html
https://www.topcv.vn/viec-lam/chuyen-vien-hau-ky-am-thanh-music-producer/1367786.html
https://www.topcv.vn/viec-lam/giao-dich-vien/1367790.html
https://www.topcv.vn/viec-lam/truong-phong-nhan-su/1367791.html
https://www.topcv.vn/viec-lam/nhan-vien-kiem-soat-noi-bo/1367792.html
https://www.topcv.vn/viec-lam/cv-cvcc-khach-hang-ca-nhan-ha-noi/1367793.html
https://www.topcv.vn/viec-lam/nhan-vien-tuyen-sinh-truong-mam-non-quoc-te/1367798.html
https://www.topcv.vn/viec-lam/game-designer/1367801.html
https://www.topcv.vn/viec-lam/thuc-tap-sinh-nhan-su/1367802.html
https://www.topcv.vn/viec-lam/kien-truc-su-noi-that/1367803.html
https://www.topcv.vn/viec-lam/quan-ly-kinh-doanh-business-development-manager/1367807.html
https://www.topcv.vn/viec-lam/assistant-manager-training/1367811.html
https://www.topcv.vn/viec-lam/production-team-leader/1367812.html
https://ww